In [1]:
# import os
# os.chdir('../..')
# os.getcwd()

'H:\\StockMaster'

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

from sklearn.preprocessing import MinMaxScaler

import FundamentalAnalysis as fa



In [4]:
import pandas_datareader as pdr
df= pdr.get_data_yahoo('AAPL', '2000-01-01')

In [5]:
# log 정규화
def nplog(dataframe):
    
    temp = np.log(dataframe)
    
    return temp

# indicator 계산
def get_indicators(dataframe, *indicators):
    
    from Indicator import get_Bolinger_Band
    from Indicator import get_DMI
    from Indicator import get_MACD
    from Indicator import get_RSI
    
    

    BAND = get_Bolinger_Band.fnBolingerBand
    DMI = get_DMI.cal_dmi
    MACD = get_MACD.fnMACD
    RSI = get_RSI.fnRSI
    
    
    for indicator in indicators:
        
        if indicator == 'DMI':
            DMI(dataframe)
        elif indicator == 'MACD':
            MACD(dataframe)
        elif indicator == 'RSI':
            RSI(dataframe)
        elif indicator == 'BAND':
            BAND(dataframe)
    
    dataframe.dropna(inplace=True)
    
    return dataframe

# minmax scale
def make_sc_col_df(train_scdf):
    for i in train_scdf.columns:
        globals()['scaler_{}'.format(i)] = MinMaxScaler()
        print("df_{}".format(i))
        globals()["df_{}".format(i)] = train_scdf[[i]]
        globals()["df_{}".format(i)] = globals()['scaler_{}'.format(i)].fit_transform(globals()["df_{}".format(i)])
        globals()["df_{}".format(i)] = pd.DataFrame(globals()["df_{}".format(i)])
        globals()["df_{}".format(i)].index = train_scdf.index
        globals()["df_{}".format(i)].columns = [i]

    return globals()["df_{}".format(i)]

# shift data
def shift_df(*dataframes, start=1, end=3):  
    shift_num = end
    for dataframe in dataframes:
        for s in range(start, end):
            dataframe['shift_{}'.format(s)] = dataframe[dataframe.columns[0]].shift(s)
        dataframe.dropna(inplace=True)
    return dataframe

# concat dataframe
def df_concat(*dataframes):
    all_Df =  pd.concat([*dataframes], axis=1)
    all_Df.dropna(inplace=True)
    return all_Df

# drop original values
def drop_ori_cols(all_df):
    col_list = []
    for i in all_df.columns:
        if 'shift' in i:
            pass
        elif 'Close' in i:
            pass
        else:
            col_list.append(i)
    all_df.drop(col_list, axis=1, inplace=True)
    
    return all_df

# set y target & split train test
def set_split(all_df, num=1, xnum=200):
    xdata = all_df.drop('Close', axis=1)[:-num]
    ydata = all_df['Close'][num:].values
    
    x_train = xdata[:-xnum]
    x_test = xdata[-xnum:]

    ytrain = ydata[:-xnum]
    ytest = ydata[-xnum:]
    
    shift_num = 2
    x_train = np.array(x_train).reshape(len(x_train), int(len(x_train.columns)/shift_num), shift_num)
    x_test = np.array(x_test).reshape(len(x_test), int(len(x_test.columns)/shift_num), shift_num)

    
    return x_train, x_test, ytrain, ytest

# make LSTM model
def LSTMmodel():
    from tensorflow.keras.layers import LSTM 
    from tensorflow.keras.models import Sequential 
    from tensorflow.keras.layers import Dense 
    import tensorflow.keras.backend as K 
    from tensorflow.keras.callbacks import EarlyStopping


    model3 = Sequential() # Sequeatial Model 
    model3.add(LSTM(20, input_shape=(x_train.shape[1], x_train.shape[2]))) # (timestep, feature) 
    model3.add(Dense(1)) # output = 1 
    model3.compile(loss='mean_squared_error', optimizer='adam') 
    model3.summary()
    
    return model3

# inverse predict 
def get_real(pred, scaler_Close):
    
    real = np.exp(scaler_Close.inverse_transform(pred))
    real = pd.DataFrame(real, columns = pred.columns)
    real['err'] = real['real'] - real['pred']
    
    return real

# model save
def save(model, name):
    # 전체 모델을 HDF5 파일로 저장합니다
    model.save('./src/parkjunsung/models/' + str(name) + '.h5')
    
# shift close
def future_stock(dataframe):
    for s in range(1, 2):
        dataframe['shift_{}'.format(s)] = dataframe[dataframe.columns[0]].shift(s)
    return dataframe

In [9]:
log_df = nplog(df)
indicators = get_indicators(log_df, 'DMI', 'RSI', 'MACD', 'BAND')
make_sc_col_df(indicators)

df_High
df_Low
df_Open
df_Close
df_Volume
df_Adj Close
df_PDI
df_MDI
df_ADX
df_RSI_MACD
df_EMAFast
df_EMASlow
df_MACD
df_MACDSignal
df_MACDDiff
df_20d_ma
df_Bol_upper
df_Bol_lower


,Bol_lower
Date,
2000-02-17,0.242594
2000-02-18,0.242593
2000-02-22,0.243110
2000-02-23,0.242967
2000-02-24,0.243023
...,...
2020-07-06,0.997545
2020-07-07,0.998926
2020-07-08,0.998954


In [6]:
new_model = load_model('./src/parkjunsung/models/stock_master.h5')
new_model.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 20)                1840      
_________________________________________________________________
dense (Dense)                (None, 1)                 21        
Total params: 1,861
Trainable params: 1,861
Non-trainable params: 0
________________

In [14]:
target = future_stock(df[['Close']])[-1:]
target

,Close,shift_1
Date,,
2020-07-10,383.679993,383.01001


In [177]:
logg = nplog(target)
sc_test = scaler_Close.transform(logg)
sc_test = sc_test.reshape(1, sc_test.shape[0], sc_test.shape[1])
real = np.exp(scaler_Close.inverse_transform(new_model.predict(sc_test)))
real

array([[495.8316]], dtype=float32)

In [178]:
pred_future = pd.DataFrame(real, columns=['Close'], index=['2020-07-14'])
pred_future = pd.concat([target, pred_future], axis=0).drop(['shift_1'], axis=1)

In [179]:
target = future_stock(pred_future)[-1:]
target

,Close,shift_1
2020-07-14,495.831604,495.134613


In [137]:
logg = nplog(target)
sc_test = scaler_Close.transform(logg)
sc_test = sc_test.reshape(1, sc_test.shape[0], sc_test.shape[1])
real = np.exp(scaler_Close.inverse_transform(new_model.predict(sc_test)))
real

array([[487.85056]], dtype=float32)